In [1]:
# train_cnn_classifier.py
import os
import json
import time
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv1D, MaxPooling1D,
                                     Dense, Dropout, BatchNormalization,
                                     Reshape, GlobalAveragePooling1D)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (EarlyStopping, ReduceLROnPlateau, 
                                      ModelCheckpoint)
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载数据集（保持原结构）"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
        print("⚠️ 未找到干扰类型名称，使用默认值")
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
    else:
        type2label = {
            "satellite_signal": 0,
            "single_tone": 1,
            "comb_spectra": 2,
            "sweeping": 3,
            "pulse": 4,
            "frequency_hopping": 5,
            "noise_fm": 6,
            "noise_am": 7,
            "random_combination": 8
        }
    
    label2name = {i: type_names[k] for k, i in type2label.items()}
    
    dataset.update({
        "type2label": type2label,
        "label2name": label2name
    })
    return dataset

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据（仅分类任务）"""
    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]])
    
    # 数据集分割（保持分层抽样）
    X_train, X_test, y_train, y_test = train_test_split(
        signals, dataset["labels"],
        test_size=0.3, random_state=42, 
        stratify=dataset["labels"]
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_test, y_test,
        test_size=0.5, random_state=42,
        stratify=y_test
    )
    
    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_train": y_train, "y_val": y_val, "y_test": y_test,
        "label2name": dataset["label2name"],
        "L": dataset["L"]
    }

# ---------- CNN分类模型 ----------
def build_cnn_model(input_shape, num_classes):
    """构建纯CNN分类模型"""
    inputs = Input(shape=input_shape)
    x = Reshape((input_shape[0], 1))(inputs)  # [batch, length, 1]
    
    # CNN特征提取
    x = Conv1D(32, 7, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(64, 5, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    x = Conv1D(128, 3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(2)(x)
    
    # 全局特征
    x = GlobalAveragePooling1D()(x)
    
    # 分类头
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    model.compile(
        optimizer=Adam(1e-3),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    return model

# ---------- 数据增强 ----------
@tf.function
def aug_fn(x):
    """信号增强"""
    x = tf.cast(x, tf.float32)
    if tf.random.uniform([]) > 0.2:
        snr = tf.random.uniform([], 5., 25.)
        noise = tf.random.normal(tf.shape(x)) * tf.math.reduce_std(x) * 10.0**(-snr/20.0)
        x = x + noise
    if tf.random.uniform([]) > 0.3:
        shift = tf.random.uniform([], -100, 100, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=0)
    if tf.random.uniform([]) > 0.3:
        scale = tf.random.uniform([], 0.7, 1.3)
        x = x * scale
    return x

# ---------- 训练单个模型 ----------
def train_single_model(data, model_idx=0, epochs=120, batch=128):
    """训练单个CNN分类模型"""
    model = build_cnn_model((data["L"],), len(data["label2name"]))
    
    # 类别权重（处理不平衡）
    cls_weights = compute_class_weight('balanced', 
                                     classes=np.unique(data['y_train']), 
                                     y=data['y_train'])
    sample_weights = np.array([cls_weights[lab] for lab in data['y_train']])
    
    # 数据管道
    train_ds = tf.data.Dataset.from_tensor_slices(
        (data['X_train'], data['y_train'], sample_weights)
    )
    train_ds = train_ds.map(lambda x, y, w: (aug_fn(x), y, w))
    train_ds = train_ds.shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE)
    
    val_ds = tf.data.Dataset.from_tensor_slices(
        (data['X_val'], data['y_val'])
    ).batch(batch).prefetch(tf.data.AUTOTUNE)
    
    # 回调函数
    os.makedirs("models", exist_ok=True)
    ckpt = f"models/cnn_classifier_{model_idx}.keras"
    callbacks = [
        EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=20, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=10),
        ModelCheckpoint(ckpt, save_best_only=True, monitor='val_sparse_categorical_accuracy')
    ]
    
    print(f"\n🔥 训练CNN分类模型 {model_idx + 1}...")
    start_time = time.time()
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    end_time = time.time()
    training_time = end_time - start_time
    print(f"✅ 模型训练完成，耗时 {training_time:.2f} 秒")
    
    return model, training_time

# ---------- 主函数 ----------
def main():
    os.makedirs("models", exist_ok=True)
    os.makedirs("visualizations", exist_ok=True)
    
    print("=" * 80)
    print("🚀 开始训练CNN干扰分类模型")
    print("=" * 80)

    # 加载数据
    print("⏳ 加载数据集...")
    dataset = load_dataset("/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz")
    data = preprocess_data(dataset)

    # 训练参数
    n_models = 1  # 训练单个模型（如需集成可改为3）
    epochs = 120
    batch_size = 128

    # 训练模型
    model, training_time = train_single_model(data, model_idx=0, epochs=epochs, batch=batch_size)

    # 评估测试集
    test_loss, test_acc = model.evaluate(data['X_test'], data['y_test'], verbose=0)
    print(f"\n📊 测试集准确率: {test_acc:.4f}")

    # 保存模型
    model.save("models/cnn_classifier_final.keras")
    print("\n✅ 训练完成！模型已保存至 models/cnn_classifier_final.keras")

if __name__ == "__main__":
    main()

2025-10-21 16:37:30.690459: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-21 16:37:30.731162: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-21 16:37:31.300378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 开始训练CNN干扰分类模型
⏳ 加载数据集...


2025-10-21 16:37:50.868823: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 44081 MB memory:  -> device: 0, name: NVIDIA vGPU-48GB, pci bus id: 0000:b8:00.0, compute capability: 8.9



🔥 训练CNN分类模型 1...
Epoch 1/120


2025-10-21 16:37:52.534501: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [56700]
	 [[{{node Placeholder/_1}}]]
2025-10-21 16:37:52.534731: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [56700]
	 [[{{node Placeholder/_2}}]]
2025-10-21 16:37:54.726106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-10-21 16:37:54.926029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-10-21

443/443 [==============================] - ETA: 0s - loss: 1.1929 - sparse_categorical_accuracy: 0.5499

2025-10-21 16:38:03.604008: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [12150]
	 [[{{node Placeholder/_1}}]]


443/443 [==============================] - 12s 18ms/step - loss: 1.1929 - sparse_categorical_accuracy: 0.5499 - val_loss: 1.0328 - val_sparse_categorical_accuracy: 0.5697 - lr: 0.0010
Epoch 2/120
443/443 [==============================] - 7s 14ms/step - loss: 0.9522 - sparse_categorical_accuracy: 0.6271 - val_loss: 0.8418 - val_sparse_categorical_accuracy: 0.6677 - lr: 0.0010
Epoch 3/120
443/443 [==============================] - 6s 12ms/step - loss: 0.8889 - sparse_categorical_accuracy: 0.6533 - val_loss: 0.7957 - val_sparse_categorical_accuracy: 0.6960 - lr: 0.0010
Epoch 4/120
443/443 [==============================] - 8s 14ms/step - loss: 0.8374 - sparse_categorical_accuracy: 0.6725 - val_loss: 0.7384 - val_sparse_categorical_accuracy: 0.7151 - lr: 0.0010
Epoch 5/120
443/443 [==============================] - 8s 16ms/step - loss: 0.8066 - sparse_categorical_accuracy: 0.6827 - val_loss: 0.7256 - val_sparse_categorical_accuracy: 0.7145 - lr: 0.0010
Epoch 6/120
443/443 [===============

In [2]:
# evaluate_cnn_classifier.py
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, confusion_matrix, 
                            classification_report)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
import matplotlib.font_manager as fm
# ---------- 中文字体设置 ----------
def set_chinese_font():
    """安全设置中文字体"""
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                     'C:/Windows/Fonts/simhei.ttf',
                     '/System/Library/Fonts/PingFang.ttc']
        for path in font_paths:
            if os.path.exists(path):
                fm.fontManager.addfont(path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 使用系统默认中文字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path):
    """加载并处理数据集"""
    data = np.load(npz_path, allow_pickle=True)
    
    # 基础数据
    dataset = {
        "signals": data["signals"],
        "labels": data["labels"].astype(np.int32),
        "jnr_values": data["jnr_values"].astype(np.float32),
        "fs": float(data["fs"]),
        "L": int(data["L"])
    }
    
    # 处理干扰类型名称
    if "interference_type_names" in data:
        type_names = data["interference_type_names"].item() if isinstance(data["interference_type_names"], np.ndarray) else data["interference_type_names"]
    else:
        type_names = {
            "satellite_signal": "Satellite_Signal",
            "single_tone": "Single_Tone",
            "comb_spectra": "Comb_Spectra",
            "sweeping": "Sweeping-LFM",
            "pulse": "Pulse",
            "frequency_hopping": "Frequency_Hopping",
            "noise_fm": "Noise_FM",
            "noise_am": "Noise_AM",
            "random_combination": "Random_Combination"
        }
        print("⚠️ 未找到干扰类型名称，使用默认值")
    
    # 创建标签映射
    if "type_to_label" in data:
        type2label = data["type_to_label"].item() if isinstance(data["type_to_label"], np.ndarray) else data["type_to_label"]
        label2name = {int(i): str(type_names[k]) for k, i in type2label.items()}
    else:
        type2label = {
            "satellite_signal": 0,
            "single_tone": 1,
            "comb_spectra": 2,
            "sweeping": 3,
            "pulse": 4,
            "frequency_hopping": 5,
            "noise_fm": 6,
            "noise_am": 7,
            "random_combination": 8
        }
        label2name = {int(i): str(type_names[k]) for k, i in type2label.items()}
    
    dataset.update({
        "type2label": type2label,
        "label2name": label2name
    })
    return dataset

# ---------- 数据预处理 ----------
def preprocess_data(dataset):
    """预处理数据"""
    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() 
                       for s in dataset["signals"]])
    
    # 数据集分割
    X_train, X_test, y_train, y_test, jnr_train, jnr_test = train_test_split(
        signals, dataset["labels"], dataset["jnr_values"],
        test_size=0.3, random_state=42, stratify=dataset["labels"]
    )
    
    return {
        "X_test": X_test,
        "y_test": y_test,
        "jnr_test": jnr_test,
        "label2name": dataset["label2name"]
    }

# ---------- 加载模型 ----------
def load_model(model_path):
    """加载训练好的CNN模型"""
    if os.path.exists(model_path):
        try:
            model = tf.keras.models.load_model(model_path)
            print(f"✅ 成功加载模型: {model_path}")
            return model
        except Exception as e:
            print(f"❌ 加载模型失败: {e}")
            return None
    print(f"⚠️ 未找到模型: {model_path}")
    return None

# ---------- 计算JNR准确率 ----------
def calculate_jnr_accuracy(y_true, y_pred, jnr_values, jnr_range):
    """计算每个JNR下的分类准确率"""
    jnr_acc = {}
    for jnr in jnr_range:
        mask = jnr_values == jnr
        if np.sum(mask) > 0:
            jnr_acc[jnr] = accuracy_score(y_true[mask], y_pred[mask])
        else:
            jnr_acc[jnr] = np.nan
    return jnr_acc

# ---------------------- 绘制混淆矩阵 ----------------------
def plot_confusion_matrix(cm, labels, title, xlabel, ylabel, filename, dpi=150, rotate_x=False):
    """专业混淆矩阵可视化"""
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_normalized = np.nan_to_num(cm_normalized)

    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(cm_normalized,
                     annot=True,
                     fmt='.2f',
                     cmap='Blues',
                     xticklabels=labels,
                     yticklabels=labels,
                     square=True,
                     annot_kws={"size": 14})
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)

    plt.title(title, pad=20, fontsize=18)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(rotation=45 if rotate_x else 0, 
               ha='right' if rotate_x else 'center', 
               fontsize=14)
    plt.yticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi)
    plt.close()

# ---------- 绘制JNR准确率曲线 ----------
def plot_jnr_accuracy(jnr_acc, filename):
    """绘制JNR-准确率曲线"""
    plt.figure(figsize=(10, 6))
    valid_jnr = [j for j in jnr_acc if not np.isnan(jnr_acc[j])]
    valid_acc = [jnr_acc[j] for j in valid_jnr]
    
    plt.plot(valid_jnr, valid_acc, 'o-', linewidth=2, markersize=8)
    plt.xlabel('JNR (dB)', fontsize=12)
    plt.ylabel('分类准确率', fontsize=12)
    plt.title('不同JNR下的干扰识别准确率', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(list(jnr_acc.keys()))
    plt.ylim(0, 1.05)
    plt.savefig(filename, dpi=300)
    plt.close()

# ---------- 主评估函数 ----------
def evaluate(model_path="models/cnn_classifier_final.keras", 
             npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"):
    """主评估流程"""
    os.makedirs("visualizations", exist_ok=True)
    os.makedirs("reports", exist_ok=True)
    
    # 1. 加载数据
    print("⏳ 加载数据集...")
    dataset = load_dataset(npz_path)
    data = preprocess_data(dataset)
    
    # 2. 加载模型
    print("⏳ 加载模型...")
    model = load_model(model_path)
    if model is None:
        print("❌ 无法加载模型，评估终止")
        return
    
    # 3. 预测
    print("⏳ 进行预测...")
    y_pred = np.argmax(model.predict(data["X_test"], verbose=1), axis=1)
    
    # 4. 计算JNR准确率
    jnr_range = np.arange(-10, 31, 5)  # [-10, -5, 0, 5, ..., 30]
    jnr_acc = calculate_jnr_accuracy(data["y_test"], y_pred, data["jnr_test"], jnr_range)
    
    # 5. 生成混淆矩阵
    print("⏳ 生成混淆矩阵...")
    cm = confusion_matrix(data["y_test"], y_pred)
    class_names = [data["label2name"][i] for i in sorted(data["label2name"].keys())]
    plot_confusion_matrix(
        cm=cm,
        labels=class_names,
        title="CNN Classification Confusion Matrix",
        xlabel="Predicted",
        ylabel="True",
        filename="visualizations/cnn_confusion_matrix.png",
        dpi=300,
        rotate_x=True
    )
    
    # 6. 绘制JNR准确率曲线
    print("⏳ 生成JNR准确率曲线...")
    plot_jnr_accuracy(jnr_acc, "visualizations/cnn_jnr_accuracy.png")
    
    # 7. 保存评估报告
    print("⏳ 生成评估报告...")
    report = {
        "overall_accuracy": float(accuracy_score(data["y_test"], y_pred)),
        "jnr_accuracy": {int(j): float(acc) if not np.isnan(acc) else None 
                         for j, acc in jnr_acc.items()},
        "confusion_matrix": cm.tolist(),
        "class_names": class_names,
        "classification_report": classification_report(
            data["y_test"], y_pred, 
            target_names=class_names,
            output_dict=True
        )
    }
    
    with open("reports/cnn_classification_report.json", "w", encoding="utf-8") as f:
        json.dump(report, f, indent=4, ensure_ascii=False)
    
    # 8. 打印结果
    print("\n" + "="*50)
    print("📊 评估结果摘要")
    print("="*50)
    print(f"整体分类准确率: {report['overall_accuracy']:.4f}")
    
    print("\n各JNR下的分类准确率:")
    for jnr in sorted(jnr_acc.keys()):
        acc = jnr_acc[jnr]
        acc_str = f"{acc:.4f}" if not np.isnan(acc) else "N/A"
        print(f"  JNR={jnr}dB: {acc_str}")
    
    print("\n✅ 评估完成！结果已保存至 reports/cnn_classification_report.json")

# ---------- 主程序 ----------
if __name__ == "__main__":
    evaluate(
        model_path="models/cnn_classifier_final.keras",
        npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"
    )

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
⏳ 加载数据集...
⏳ 加载模型...


2025-10-21 23:09:21.703594: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 895 MB memory:  -> device: 0, name: NVIDIA vGPU-48GB, pci bus id: 0000:d8:00.0, compute capability: 8.9


✅ 成功加载模型: models/cnn_classifier_final.keras
⏳ 进行预测...
  1/760 [..............................] - ETA: 4:22

2025-10-21 23:09:22.123736: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-10-21 23:09:22.276764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


760/760 [==============================] - 2s 2ms/step
⏳ 生成混淆矩阵...
⏳ 生成JNR准确率曲线...
⏳ 生成评估报告...

📊 评估结果摘要
整体分类准确率: 0.8269

各JNR下的分类准确率:
  JNR=-10dB: 0.4066
  JNR=-5dB: 0.5908
  JNR=0dB: 0.7669
  JNR=5dB: 0.8595
  JNR=10dB: 0.9250
  JNR=15dB: 0.9587
  JNR=20dB: 0.9777
  JNR=25dB: 0.9781
  JNR=30dB: 0.9738

✅ 评估完成！结果已保存至 reports/cnn_classification_report.json


CNN整体分类准确率: 0.8269

各JNR下的分类准确率:
  JNR=-10dB: 0.4066
  JNR=-5dB: 0.5908
  JNR=0dB: 0.7669
  JNR=5dB: 0.8595
  JNR=10dB: 0.9250
  JNR=15dB: 0.9587
  JNR=20dB: 0.9777
  JNR=25dB: 0.9781
  JNR=30dB: 0.9738